In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Foodie_CLIP

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Foodie_CLIP


In [ ]:
!pip install transformers
!pip install torcheval
!pip install torch
!pip install accelerate -U

In [ ]:
from foodie_clip.foodieclip import FoodieClip, ClIPLoss
from trainer.utils import CLIPDataCollator, CLIPDataset
from trainer.evaluate import compute_accuracy
from foodie_clip.foodieclip import FoodieClip, ClIPLoss
from trainer.utils import CLIPDataset, CLIPDataCollator
from trainer.clip_trainer import CLIPTrainer
from transformers import TrainingArguments, AutoProcessor, CLIPConfig

In [ ]:
# loading model
# loading model
foodieclip_config = CLIPConfig.from_pretrained("openai/clip-vit-base-patch32")
foodieclip = FoodieClip(foodieclip_config, projection_dim=128).cuda()
# loading dataset
train_dataset = CLIPDataset(train=True,path="/content/drive/MyDrive/Colab Notebooks/Foodie_CLIP/", class_max_datapoints=200)
print(len(train_dataset))
eval_dataset = CLIPDataset(train=False, path="/content/drive/MyDrive/Colab Notebooks/Foodie_CLIP/", class_max_datapoints=100)
# set data collator
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")
data_collator = CLIPDataCollator(processor=processor)
# set up training args
training_args = TrainingArguments(
    output_dir = "output/",
    per_device_train_batch_size=500,
    evaluation_strategy="steps",
    logging_steps=1,
    num_train_epochs = 2,
    report_to=None,
    eval_steps = 5,
    fp16=False,
    remove_unused_columns=False
)
trainer = CLIPTrainer(
    model=foodieclip,
    training_args = training_args,
    data_collator=data_collator,
    max_length=128,
    compute_metrics=compute_accuracy,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    Loss_FN=ClIPLoss
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


47177


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,caption match accuracy
5,5.811000,1.112246,[0.9518588]
10,5.785800,1.079295,[0.9494603]
15,5.802300,1.077226,[0.95134485]


Trainer is attempting to log a value of "[0.9518588]" of type <class 'numpy.ndarray'> for key "eval/ caption match accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


...

Trainer is attempting to log a value of "[0.9494603]" of type <class 'numpy.ndarray'> for key "eval/ caption match accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


[0.9494603]


Trainer is attempting to log a value of "[0.95134485]" of type <class 'numpy.ndarray'> for key "eval/ caption match accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


[0.95134485]


KeyboardInterrupt: 